In [ ]:
import json
from IPython.display import clear_output, HTML, display
import ipywidgets.widgets as widgets
# from tqdm.notebook import tqdm

import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')

import ballsbot.config as config
config.ENABLE_MULTIPROCESSING = False

In [ ]:
import ballsbot.drawing as drawing
from ballsbot.lidar import calibration_to_xywh
from ballsbot.drawing import BotPoseAbsCoordsDrawing
from ballsbot.utils import keep_rps
from ballsbot.lidar import revert_transformation_to_cloud
from ballsbot.dashboard import Dashboard

In [ ]:
only_nearby_meters = 4

In [ ]:
new_format = True
with open('/home/ballsbot/projects/ballsbot/data/track-2024-07-11.json', 'r') as hf:
    if new_format:
        track = []
        for line in hf:
            try:
                frame = json.loads(line)
                track.append(frame)
            except Exception as e:
                print(f'broken frame {e}')
    else:
        track = json.loads(hf.read())

In [ ]:
calibration = {
    'angle_fix': 3.0810609316737434,  # lidar orientation (radians)
    'fl_x': 0.2041686441507201,  # front left corner coords
    'fl_y': 0.10426277741236079,
    'rr_x': -0.08011094659163859,  # rear right corner coords
    'rr_y': -0.0988675829003773,
}
self_position = calibration_to_xywh(calibration)


dashboard = Dashboard(widgets, image_size=800)

augmented_painter = BotPoseAbsCoordsDrawing(dashboard, 'augmented')
dashboard.add_subplot('augmented', augmented_painter.get_drawing_func_name())
sparse_painter = BotPoseAbsCoordsDrawing(dashboard, 'sparse')
dashboard.add_subplot('sparse', sparse_painter.get_drawing_func_name())

display(dashboard.get_image())

In [ ]:
ts = None
for frame_index, track_frame in enumerate(track):
# for frame_index, track_frame in enumerate(tqdm(track)):
    if not track_frame['tracker']['current_pose']:
        continue
    ts = keep_rps(ts, fps=1)
    pose = {'ts': track_frame['tracker']['current_pose']['ts'], 'x': 0., 'y': 0., 'teta': 0.}

    image_text = json.dumps(track_frame['direction'], sort_keys=True)
    if track_frame['lidar']['augmented_points']:
        augmented_painter.update_image(
            [pose],
            track_frame['lidar']['augmented_points'],
            self_position,
            image_text=image_text,
            only_nearby_meters=only_nearby_meters
        )
    if track_frame['lidar']['sparse_points']:
        if track_frame['tracker']['free_tile_centers']:
            cells = [track_frame['tracker']['target_point']] + track_frame['tracker']['free_tile_centers']
            transformation = (track_frame['tracker']['current_pose']['x'], track_frame['tracker']['current_pose']['y'], track_frame['tracker']['current_pose']['teta'])
            transformed_cells = revert_transformation_to_cloud(cells, transformation)
            free_cells = transformed_cells[1:]
            target_point = transformed_cells[0]
        else:
            free_cells = None
            target_point = None
        
        sparse_painter.update_image(
            [pose],
            track_frame['lidar']['sparse_points'],
            self_position,
            free_cells=free_cells,
            target_point=target_point,
            image_text=image_text,
            only_nearby_meters=only_nearby_meters
        )
        
    dashboard.redraw()

    clear_output(wait=True)
    print(frame_index)
    print(json.dumps(track_frame['directions_weights'], indent=4, sort_keys=True))